In [1]:
import folium
import pandas as pd

In [2]:
from folium.plugins import MarkerCluster
from folium.plugins import  MousePosition
from folium.features import DivIcon

In [3]:
df = pd.read_csv('dataset_part_2.csv')

In [15]:
world_map = folium.Map(location=[df.Latitude.mean(), df.Longitude.mean()], zoom_start=5)


In [21]:
instance = folium.FeatureGroup(name='Launch Sites')
for lat, lng in zip(df.Latitude, df.Longitude):
    instance.add_child(
        folium.CircleMarker(
            location=[lat, lng],
            radius=5,
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

In [22]:
world_map.add_child(instance)

In [20]:
import folium
launch_sites = {
    'CCAFS SLC 40': [28.562302, -80.577356],
    'VAFB SLC 4E': [34.632834, -120.610746],
    'KSC LC 39A': [28.573255, -80.646895],
    'CCAFS SLC 40': [28.562302, -80.577356]
    # Add other sites if any
}

# Initialize map centered roughly at Florida launch sites
map_ = folium.Map(location=[28.562302, -80.577356], zoom_start=5)

# Add markers for each launch site
for site, coords in launch_sites.items():
    folium.Marker(location=coords, popup=site).add_to(map_)

# Display the map in Jupyter Notebook
map_


In [24]:
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

In [25]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [30]:
spacex_df = spacex_df[['Launch Site','Lat','Long','class']]
launch_sites_df = spacex_df.groupby(['Launch Site'],as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site','Lat','Long']]

In [31]:
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [32]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [34]:
site_map

In [35]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Cre
# ate a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    iconmarker = DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)
site_map.add_child(circle)
site_map.add_child(marker)

In [36]:
import folium

# Initialize the map at NASA coordinate with zoom 5
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each launch site in launch_sites_df
for _, row in launch_sites_df.iterrows():
    lat = row['Lat']
    lon = row['Long']
    site_name = row['Launch Site']
    
    # Create a Circle for each launch site with popup label
    circle = folium.Circle(
        location=[lat, lon],
        radius=1000,            # radius in meters
        color='#d35400',
        fill=True,
        fill_opacity=0.5
    ).add_child(folium.Popup(site_name))
    
    # Add the circle to the map
    site_map.add_child(circle)

# Display the map (in Jupyter Notebook)
site_map


In [37]:
import folium

# Initialize the map (centered at NASA coord or average of all sites)
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each launch record (each row is one launch)
for _, row in spacex_df.iterrows():
    lat = row['Lat']
    lon = row['Long']
    site_name = row['Launch Site']
    launch_status = row['class']  # 1 or 0
    
    # Choose marker color based on success/failure
    if launch_status == 1:
        marker_color = 'green'   # success: green
        popup_text = f"{site_name} - Success"
    else:
        marker_color = 'red'     # failure: red
        popup_text = f"{site_name} - Failure"
    
    # Add marker with colored icon and popup
    folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        icon=folium.Icon(color=marker_color)
    ).add_to(site_map)

# Display the map
site_map


In [38]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [39]:
marker_cluster = MarkerCluster()

In [41]:
marker_colors=[]
for status in spacex_df['class']:
    if status==1:
        marker_colors.append('green')
    else:
        marker_colors.append('red')
spacex_df['marker_color']  = marker_colors           

In [42]:

for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.Marker(
        location=[record['Lat'],record['Long']],
           icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)
site_map.add_child(marker_cluster)
site_map

In [47]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [45]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance